In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

### Creating model

In [ ]:
#intialising hyperparameters
in_channels = 3
num_classes = 2

batch_size = 64
learning_rate = 0.01

linear_first_dimension = 2048
num_epochs = 3

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=in_channels, num_classes=num_classes):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels, 8, kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.cnn2 = nn.Conv2d(8, 16, kernel_size=(3,3), padding=(1,1), stride=(1,1))
#         self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.cnn3 = nn.Conv2d(16,32, kernel_size=(3,3), padding=(1,1),stride=(1,1))
        self.fc1 = nn.Linear(linear_first_dimension,num_classes)
    
    def forward(self, x):
        out = self.pool(self.cnn1(x))
        out = self.pool(self.cnn2(out))
        out = self.pool(self.cnn3(out))
        out = out.reshape(out.size(0), -1)
#         print(out.shape)
        out = self.fc1(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = CNN().to(device=device)

#testing
x = torch.randn(64,3,64,64).to(device=device)
y = model(x)
y.shape

### Working with Dataset

In [ ]:
import matplotlib.pyplot as plt
img = Image.open('../input/celeba-dataset/img_align_celeba/img_align_celeba/000022.jpg')
plt.imshow(img)

In [ ]:
img._size

In [ ]:
import pandas as pd
df = pd.read_csv('../input/celeba-dataset/list_attr_celeba.csv')
df.head()

In [ ]:
a = df[['Attractive','image_id']]
a.head()

In [ ]:
a.iloc[:,0]

In [ ]:
a['target'] = [0 if x==-1 else 1 for x in a.iloc[:,0]]

In [ ]:
a.head()

In [ ]:
imagepath = "../input/celeba-dataset/img_align_celeba/img_align_celeba"
csvpath = "../input/celeba-dataset/list_attr_celeba.csv"

In [ ]:
import os

In [ ]:
os.path.join(imagepath,a['image_id'][0])

In [ ]:
my_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.Resize((64,64)),
])

In [ ]:
img = my_transforms(img)
img.size

In [ ]:
plt.imshow(img)

### Creating pytorch Dataset class

In [ ]:
class CelebAttractivenessDataset(Dataset):
    def __init__(self, imagepath=imagepath, csvpath=csvpath, transform=None):
        df = pd.read_csv(csvpath)
        a = df[['Attractive','image_id']]
        a['target'] = [0 if x==-1 else 1 for x in a.iloc[:,0]]
        self.y = a['target']
        self.x = a['image_id']
        self.n_samples = len(a)
        self.transform = transform
        
    def __getitem__(self,index):
        img = Image.open(os.path.join(imagepath, self.x[index])).convert('RGB')
        if self.transform:
            img = self.transform(img)
            
#         img_tensor = np.array(img)
        return transforms.functional.to_tensor(img), self.y[index]
    
    def __len__(self):
        return self.n_samples

In [ ]:
train_dataset = CelebAttractivenessDataset(transform=my_transforms)
img, y = train_dataset[0]

plt.imshow(img.permute(1,2,0))
print(y)

In [ ]:
img.shape

### Creating a DataLoader

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

In [ ]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# training
for epoch in range(num_epochs):
    for data, target in train_dataloader:
        data = data.to(device=device)
#         data = data.permute(0,3,1,2)
        target = target.to(device=device)
        
        score = model(data)
        loss = loss_criterion(score, target)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    print(f"For epoch : {epoch} loss: {loss}")

In [ ]:
#checking accuracy helper function
def check_accuracy(model, loader):
    total_correct = 0
    total_predictions = 0
    
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x= x.to(device=device)
            y= y.to(device=device)
            y_pred = model(x)
            _, prediction = y_pred.max(1)
        
            total_correct +=  (y==prediction).sum()
            total_predictions += prediction.shape[0]
    
    print(f" the accuracy: {(total_correct/total_predictions)*100}")
    model.train()
        

In [ ]:
check_accuracy(model, train_dataloader)
check_accuracy(model, test_dataloader)